In [1]:
# 0 0 * * * /opt/anaconda3/envs/jupyter_env/bin/ipython /home/slebedev/jhub-analytics/'_ Analytics department'/'[АВТОЗАПУСК]'/'[АВТОЗАПУСК] BA-470 Оплаченные заказы с сайта для метрики.py'

# magic    
get_ipython().run_line_magic('matplotlib', 'inline')
# run files
get_ipython().run_line_magic('run', '/srv/jhub/share/._Additions/import_libs.py')
get_ipython().run_line_magic('run', '/srv/jhub/share/._Additions/Connecting_.py')
get_ipython().run_line_magic('run', '/srv/jhub/share/._Additions/my_script.py')
get_ipython().run_line_magic('run', '/srv/jhub/share/._Additions/Date_gen.py')
get_ipython().run_line_magic('run', '/srv/jhub/share/._Additions/xx.py')

############################## End ##############################

In [2]:
import requests
import sys
import json
import pandas as pd
import numpy as np
import datetime
import time
import pytz

# Функции для метрики
from Metrika_logs import param_logs
from Metrika_logs import make_logs
from Metrika_logs import param_costs
from Metrika_logs import report_costs

# Функции для БД
from Q478_sql import db_connect
from Q478_sql import query
from Q478_sql import sql_query

1. Выгружаем данные из Я.Метрики за отчетный период

In [14]:
# Определяем период для отчета

report_date = datetime.datetime.now() - datetime.timedelta(days=9)

date_from = (report_date.date() - datetime.timedelta(days=29)).strftime("%Y-%m-%d")
date_till = report_date.date().strftime("%Y-%m-%d") # Дата окончания отчета

# Рекламные аккаунты Я.Директа

client_logins = ['e2e4online18', 'e2e4vendor', 'e2e4gender']

# Определяем цели метрики

targets = {'Оформление заказа на сайте': 45333839, 'CRM: Заказ создан': 296990514}#, 'CRM: Заказ оплачен': 296990523}


In [16]:
date_till

'2023-08-14'

In [15]:
date_from

'2023-07-16'

In [17]:
# Получаем параметры запроса для выгрузки логов

metrika_headers, metrika_url, payload, count, field_rus = param_logs(date_from, date_till)

# Выгружаем логи с сервера Яндекса в датасет

df_logs = make_logs(metrika_headers, metrika_url, payload, count, field_rus)

1. Проверяем возможность создания логов на сервере Яндекса
2. Отправляем запрос на создание логов на сервер Яндекса
3. Проверяем статус запроса 31764443 на сервере Яндекса
4. Логи готовы (2 партии), скачиваем их с сервера Яндекса
5. Логи скачены, удаляем их с сервера Яндекса
6. Логи удалены с сервера Яндекса


In [18]:
df_log_test = df_logs.copy()

In [19]:
# Преобразование типа данных полей таблицы Я.Метрики 

df_logs['Время достижения цели'] = df_logs['Время достижения цели'].str.replace("\\", "")

for i in ['ID целей',
        'Время достижения цели',
        'ID заказов']:
    df_logs[i] = df_logs[i].apply(lambda x: eval(x))

# Фильтруем уникальные позиции заказов 

def unique_ord(row):
    un = set(row)
    l = list(un)
    try:
        l.remove('')
    except:
        l
    return l

df_logs['ID заказов'] = df_logs['ID заказов'].apply(unique_ord)

# Оставляем только целевые визиты

for key, value in targets.items():
    df_logs[key] = df_logs['ID целей'].apply(lambda x: 'order' if value in x else np.nan)
    
df_logs = df_logs[df_logs['Оформление заказа на сайте'].notna() | df_logs['CRM: Заказ создан'].notna()]# | df_logs['CRM: Заказ оплачен'].notna()]
df_logs.drop(columns=['Оформление заказа на сайте', 'CRM: Заказ создан'], inplace=True)

# Выделяем цели ЯМ, касающиеся только оформления заказа на сайте

df_logs = df_logs.explode(['ID целей', 'Время достижения цели'])
df_logs = df_logs[df_logs['ID целей'].isin(list(targets.values()))]

# Приводим даты и время к часовому поясу НСК+7

tz_3 = pytz.timezone("Europe/Moscow")
tz_7 = pytz.timezone("Asia/Novosibirsk")

def interval(row):
    lst = []
    for i in range(-3, 6):
        x = datetime.datetime.strftime(tz_3.localize(datetime.datetime.strptime(row, '%Y-%m-%d %H:%M:%S')).astimezone(tz_7) - datetime.timedelta(seconds=i), '%Y-%m-%d %H:%M:%S')
        lst.append(x)
    return lst

df_logs['interval'] = df_logs['Время достижения цели'].apply(interval)

for i in ['Дата и время визита', 'Время достижения цели']:
    df_logs[i] = df_logs[i].apply(lambda x: datetime.datetime.strftime(tz_3.localize(datetime.datetime.strptime(x, '%Y-%m-%d %H:%M:%S')).astimezone(tz_7), '%Y-%m-%d %H:%M:%S'))
    


/tmp/ipykernel_3334849/1837451763.py:3: FutureWarning:

The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.



In [20]:
# Выгружаем затраты с сервера Яндекса в датасет

download_status, df_costs = report_costs(date_from, date_till, client_logins)
download_status

200

In [21]:
# Преобразование таблицы расходов Я.Метрики

df_costs.columns = ['Источник трафика', 'Кампания ЯД', 'Название кампании', 'Расходы по рекламной кампании']
df_costs = df_costs.drop_duplicates()

2. Выгружаем заказы из БД за отчетный период

In [22]:
# Формируем SQL запрос

sql_query = sql_query(date_from, date_till)

In [ ]:
# Получаем данные из БД

df_sql = query(sql_query)

1. Соединение с PostgreSQL установлено. Загружаем данные...


In [170]:
# Преобразование типа данных таблицы заказов из БД

df_sql['order_date'] = df_sql['order_date'].apply(lambda x: datetime.datetime.strftime(x.astimezone(tz_7), '%Y-%m-%d %H:%M:%S'))
df_sql['y_client_ids'] = df_sql['y_client_ids'].apply(lambda x: None if x == 'null' else x)
for i in range(9, 14):
    df_sql.iloc[:, i] = df_sql.iloc[:, i].fillna(0)

AttributeError: 'str' object has no attribute 'astimezone'

3. Объединяем заказы из БД и заказы из Я.Метрики

In [198]:
# Объединяем клиентов из БД и метрики
df_merge = df_sql.merge(df_logs, how='left', left_on='y_client_ids', right_on='ID пользователя')
df_merge = df_merge[df_merge['ID пользователя'].notna()]

# Сводим заказы по времени оформления
df_merge['in'] = df_merge.apply(lambda x: x['order_date'] in x['interval'], axis=1)
df_merge = df_merge[df_merge['in']]
df_merge = df_merge[~df_merge.duplicated(subset=["id"])]

# Объединяем заказы из БД и сведенные заказы в одну таблицу
df_merge2 = df_sql.merge(df_merge, how='left', on='id')
df_merge2 = df_merge2.merge(df_costs[df_costs['Кампания ЯД'].notna()], how='left', on='Кампания ЯД')
df_merge2['Кампания ЯД'] = df_merge2['Кампания ЯД'].apply(lambda x: int(x) if x != '' and pd.notna(x) else np.nan).astype('Int64')
df_merge2['Расходы по рекламной кампании'] = df_merge2['Расходы по рекламной кампании'].fillna(0)

In [199]:
# Расходы по рекламным системам

ya_direct = sum(df_costs[df_costs['Источник трафика'] == 'Yandex.Direct']['Расходы по рекламной кампании'])
vk = sum(df_costs[df_costs['Источник трафика'] == 'vk']['Расходы по рекламной кампании']) + sum(df_costs[df_costs['Источник трафика'] == 'elama.vk_ads']['Расходы по рекламной кампании'])
retailrocket = sum(df_costs[df_costs['Источник трафика'] == 'retailrocket']['Расходы по рекламной кампании'])
news = sum(df_costs[df_costs['Источник трафика'] == '3dnews']['Расходы по рекламной кампании'])
gis = sum(df_costs[df_costs['Источник трафика'] == '2gis']['Расходы по рекламной кампании'])
admitad = sum(df_costs[df_costs['Источник трафика'] == 'admitad']['Расходы по рекламной кампании'])
regmarkets = sum(df_costs[df_costs['Источник трафика'] == 'regmarkets.ru']['Расходы по рекламной кампании'])
price_ru = sum(df_costs[df_costs['Источник трафика'] == 'priceru']['Расходы по рекламной кампании'])

def spends(row):
    if row['Рекламная система'] == 'ya_direct' or row['Рекламная система'] == 'ya_undefined' or pd.notna(row['Кампания ЯД']):
        x = round(ya_direct, 1)    
    elif row['Рекламная система'] == 'vkontakte':
        x = round(vk, 1)
    elif row['Рекламная система'] == 'admitad':
        x = round(admitad, 1)
    elif row['Рекламная система'] == 'market':
        x = round(regmarkets, 1)
    elif row['Рекламная система'] == 'price_ru':
        x = round(price_ru, 1)
    else:
        x = 0
    return x

df_merge2['Расходы по рекламному каналу'] = df_merge2.apply(spends, axis = 1)

In [200]:
# Рекламные затраты на заказ

# количество заказов 
ya_orders = len(df_merge2[((df_merge2['Рекламная система'] == 'ya_direct') | (df_merge2['Рекламная система'] == 'ya_undefined') | (pd.notna(df_merge2['Кампания ЯД'])))])    
vk_orders = len(df_merge2[(df_merge2['Рекламная система'] == 'vkontakte') & (pd.notna(df_merge2['Рекламная система'])) & (pd.isna(df_merge2['Кампания ЯД']))])
adm_orders = len(df_merge2[(df_merge2['Рекламная система'] == 'admitad') & (pd.notna(df_merge2['Рекламная система'])) & (pd.isna(df_merge2['Кампания ЯД']))])
mar_orders = len(df_merge2[(df_merge2['Рекламная система'] == 'market') & (pd.notna(df_merge2['Рекламная система'])) & (pd.isna(df_merge2['Кампания ЯД']))])
pr_orders = len(df_merge2[(df_merge2['Рекламная система'] == 'price_ru') & (pd.notna(df_merge2['Рекламная система'])) & (pd.isna(df_merge2['Кампания ЯД']))])

def spend_order(row):
    if row['Рекламная система'] == 'ya_direct' or row['Рекламная система'] == 'ya_undefined' or pd.notna(row['Кампания ЯД']):
        x = round(row['Расходы по рекламному каналу']/ya_orders, 1)    
    elif row['Рекламная система'] == 'vkontakte' and pd.isna(row['Кампания ЯД']):
        x = round(row['Расходы по рекламному каналу']/vk_orders, 1)  
    elif row['Рекламная система'] == 'admitad' and pd.isna(row['Кампания ЯД']):
        x = round(row['Расходы по рекламному каналу']/adm_orders, 1)  
    elif row['Рекламная система'] == 'market' and pd.isna(row['Кампания ЯД']):
        x = round(row['Расходы по рекламному каналу']/mar_orders, 1)  
    elif row['Рекламная система'] == 'price_ru' and pd.isna(row['Кампания ЯД']):
        x = round(row['Расходы по рекламному каналу']/pr_orders, 1)
    else:
        x = 0
    return x

df_merge2['Рекламные затраты на заказ'] = df_merge2.apply(spend_order, axis = 1)


In [201]:
# Рекламные затраты на выкупленный заказ

# количество выкупов
ya_ship_ord = len(df_merge2[((df_merge2['Рекламная система'] == 'ya_direct') | (df_merge2['Рекламная система'] == 'ya_undefined') | (pd.notna(df_merge2['Кампания ЯД']))) & (df_merge2['sum_payed_x'] != 0) & (pd.notna(df_merge2['Рекламная система']))])    
vk_ship_ord = len(df_merge2[(df_merge2['Рекламная система'] == 'vkontakte') & (df_merge2['sum_payed_x'] != 0) & (pd.notna(df_merge2['Рекламная система'])) & (pd.isna(df_merge2['Кампания ЯД']))])
adm_ship_ord = len(df_merge2[(df_merge2['Рекламная система'] == 'admitad') & (df_merge2['sum_payed_x'] != 0) & (pd.notna(df_merge2['Рекламная система'])) & (pd.isna(df_merge2['Кампания ЯД']))])
mar_ship_ord = len(df_merge2[(df_merge2['Рекламная система'] == 'market') & (df_merge2['sum_payed_x'] != 0) & (pd.notna(df_merge2['Рекламная система'])) & (pd.isna(df_merge2['Кампания ЯД']))])
pr_ship_ord = len(df_merge2[(df_merge2['Рекламная система'] == 'price_ru') & (df_merge2['sum_payed_x'] != 0) & (pd.notna(df_merge2['Рекламная система'])) & (pd.isna(df_merge2['Кампания ЯД']))])

def ship_order(row):
    if (row['Рекламная система'] == 'ya_direct' or row['Рекламная система'] == 'ya_undefined' or pd.notna(row['Кампания ЯД'])) and row['sum_payed_x'] != 0:
        x = round(row['Расходы по рекламному каналу']/ya_ship_ord, 1)    
    elif row['Рекламная система'] == 'vkontakte' and pd.isna(row['Кампания ЯД']) and row['sum_payed_x'] != 0:
        x = round(row['Расходы по рекламному каналу']/vk_ship_ord, 1)  
    elif row['Рекламная система'] == 'admitad' and pd.isna(row['Кампания ЯД']) and row['sum_payed_x'] != 0:
        x = round(row['Расходы по рекламному каналу']/adm_ship_ord, 1)  
    elif row['Рекламная система'] == 'market' and pd.isna(row['Кампания ЯД']) and row['sum_payed_x'] != 0:
        x = round(row['Расходы по рекламному каналу']/mar_ship_ord, 1)  
    elif row['Рекламная система'] == 'price_ru' and pd.isna(row['Кампания ЯД']) and row['sum_payed_x'] != 0:
        x = round(row['Расходы по рекламному каналу']/pr_ship_ord, 1)
    else:
        x = 0
    return x

df_merge2['Рекламные затраты на выкупленный заказ'] = df_merge2.apply(ship_order, axis = 1)

In [203]:
# Рассчет маржинальности

df_merge2['Маржа чистая - рекламные затраты на заказ'] = df_merge2['revenue_x'] - df_merge2['Рекламные затраты на заказ']
df_merge2['Маржа чистая - рекламные затраты на выкупленные заказы'] = (df_merge2['sum_payed_x'] - df_merge2['Рекламные затраты на выкупленный заказ']).fillna(0)

In [204]:

# подготавливаем таблицу к загрузке в облако
df = df_merge2[['id', 'order_status_x', 'db_client_ids_x', 'y_client_ids_x', 
                'order_date_x', 'revenue_x', 'item_count_x', 'marginality_x', 'sum_shipped_x', 
                'ship_item_count_x', 'sum_payed_x', 'margin_x', 'Город', 'client_sity_x', 
                'Рекламная система', 'Кампания ЯД', 'Расходы по рекламному каналу', 
                'Расходы по рекламной кампании', 'Рекламные затраты на заказ', 
                'Маржа чистая - рекламные затраты на заказ', 'Рекламные затраты на выкупленный заказ', 
                'Маржа чистая - рекламные затраты на выкупленные заказы']]

df.columns = ['ID заказа ', 'Статус заказа', 'ID клиента е2е4', 'ID клиента Яндекс', 
              'Дата оформления заказа', 'Сумма оформленного заказа', 'Кол-во позиций в заказе', 
              'Маржинальность заказа ', 'Сумма отгруженного заказа', 'Кол-во позиций отгруженного заказа ',
              'Сумма оплаты', 'Маржа чистая', 'Рекламный город', 'Город клиента', 
              'Рекламная система', 'Кампания ЯД', 'Расходы по рекламному каналу', 
              'Расходы по рекламной кампании', 'Рекламные затраты на заказ', 
              'Маржа чистая - рекламные затраты на заказ', 'Рекламные затраты на выкупленный заказ', 
              'Маржа чистая - рекламные затраты на выкупленные заказы']


4. Загружаем отчет в облачное хранилище 

In [177]:
# Создаем адрес загрузки

Amazon_owncloud = owncloud.Client('https://nextcloud.e2e4.ru')
Amazon_owncloud.login(loginAtlassianjhubAdmin, passwordAtlassianjhubAdmin)

AmazonS3_folder_Public = r'/Public/Отчеты (выгрузка jupyter notebooks)'
AmazonS3_path_to_file = r'/Рекламные затраты на заказы из метрики/'

AmazonS3_full_path_customer = AmazonS3_folder_Public + AmazonS3_path_to_file + r'Рекламные затраты на заказы из метрики'


In [205]:
df.to_excel(temp_tmp + AmazonS3_full_path_customer.split('/')[-1] + '.xlsx',  index=False, encoding='cp-1251') #   to_excel

try:
    Amazon_owncloud.put_file( AmazonS3_full_path_customer + '.xlsx', temp_tmp + AmazonS3_full_path_customer.split('/')[-1] + '.xlsx' )
    print( f"✔ Файл записан в облако\n{AmazonS3_full_path_customer.split('/')[-1]}.xlsx" )  
    os.remove(temp_tmp + AmazonS3_full_path_customer.split('/')[-1] + '.xlsx' )
    print( '✔ os.remove' )
except:
    print( 'Возникли проблемы при записи в облако!' )

✔ Файл записан в облако
Рекламные затраты на заказы из метрики.xlsx
✔ os.remove
